In [6]:
#Importing required libraries
import nltk
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [7]:
#reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [16]:
#Splitting the corpus into train and test,checking the lenght of each set and printing the first 10 sentences in train set
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

print(len(train_set))
print(len(test_set))
print(train_set[:10])

3718
196
[[('January', 'NOUN'), ('platinum', 'NOUN'), ('was', 'VERB'), ('down', 'ADP'), ('$', '.'), ('5.70', 'NUM'), ('*U*', 'X'), ('an', 'DET'), ('ounce', 'NOUN'), ('at', 'ADP'), ('$', '.'), ('494.50', 'NUM'), ('*U*', 'X'), ('.', '.')], [('The', 'DET'), ('spokesman', 'NOUN'), ('said', 'VERB'), ('0', 'X'), ('it', 'PRON'), ('is', 'VERB'), ('the', 'DET'), ('first', 'ADV'), ('early', 'ADJ'), ('retirement', 'NOUN'), ('plan', 'NOUN'), ('offered', 'VERB'), ('under', 'ADP'), ('its', 'PRON'), ('two-year', 'ADJ'), ('cost-control', 'ADJ'), ('strategy', 'NOUN'), ('.', '.')], [('To', 'PRT'), ('the', 'DET'), ('astonishment', 'NOUN'), ('and', 'CONJ'), ('dismay', 'NOUN'), ('of', 'ADP'), ('her', 'PRON'), ('superiors', 'NOUN'), ('and', 'CONJ'), ('legal', 'ADJ'), ('authorities', 'NOUN'), ('--', '.'), ('and', 'ADJ'), ('perhaps', 'ADV'), ('as', 'ADP'), ('a', 'DET'), ('measure', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('unpopularity', 'NOUN'), ('of', 'ADP'), ('standardized', 'ADJ'), ('tests', 'NOUN'), ('--

In [17]:
# Get the list of tagged words from train set
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95478

In [19]:
len(vocab)

NameError: name 'vocab' is not defined

In [18]:
def viterbi(vocab, vocab_tag, words, tags = train_tagged_words, t_bigram_count, t_unigram_count, e_bigram_count, e_unigram_count, ADD_K):
    vocab_size = len(vocab)
    V = [{}]

    for t in vocab_tag:
        # Prob of very first word
        prob = np.log2(float(e_bigram_count.get((words[0],t),0)+ADD_K))-np.log2(float(e_unigram_count[t]+vocab_size*ADD_K))
        # trigram V[0][0]
        V[0][t] = {"prob": prob, "prev": None}

    for i in range(1,len(words)):
        V.append({})
        for t in vocab_tag:
            V[i][t] =  {"prob": np.log2(0), "prev": None}
        for t in vocab_tag:
            max_trans_prob = np.log2(0);
            for prev_tag in vocab_tag:
                trans_prob = np.log2(float(t_bigram_count.get((t, prev_tag),0)+ADD_K))-np.log2(float(t_unigram_count[prev_tag]+vocab_size*ADD_K))   
                if V[i-1][prev_tag]["prob"]+trans_prob > max_trans_prob:
                    max_trans_prob = V[i-1][prev_tag]["prob"]+trans_prob 
                    max_prob = max_trans_prob+np.log2(e_bigram_count.get((words[i],t),0)+ADD_K)-np.log2(float(e_unigram_count[t]+vocab_size*ADD_K))
                    V[i][t] = {"prob": max_prob, "prev": prev_tag}
    opt = []
    previous = None 
    max_prob = max(value["prob"] for value in V[-1].values())
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            opt.append(st)
            previous = st
            break
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t][previous]["prev"]
    return opt

SyntaxError: non-default argument follows default argument (<ipython-input-18-41c0f5b3716d>, line 1)